In [1]:
import pandas as pd
data_train = pd.read_csv("data_train.csv", header=0)

In [2]:
#on va ajouter des distances dans le calcul du KNN. On se focalisera pour l'instant sur les distances qui vectorisent les chaînes de caractères à l'échelle de mots, ou à l'échelle des caractères.

In [23]:
from pytextdist.edit_distance import * #distances calculées à l'échelle du caractères
from pytextdist.vector_similarity import * #distances calculées en vectorisant les chaînes de caractères.Ici en mots, mais configurable pour des n-grammes.


class KNN():
    
    def __init__(self, data, tweet_a_categoriser, nombre_voisins, distance, vote):
        self.data = data
        self.tweet_a_categoriser = tweet_a_categoriser
        self.nombre_voisins = nombre_voisins
        self.distance = distance
        self.vote = vote
        
        
    def liste_distance(self):
        distances = []
        
        if self.distance == "naïve":
            mots_tweet = self.tweet_a_categoriser.split()
            nombre_de_mots_tweet = len(mots_tweet)
            for autre_tweet in self.data["text"]:
                mots_autre_tweet = autre_tweet.split()
                nombre_de_mots_autre_tweet = len(mots_autre_tweet)
                nombre_total_mots = nombre_de_mots_tweet + nombre_de_mots_autre_tweet

                mots_communs = set()
                for mot in mots_tweet:
                    if mot in mots_autre_tweet:
                        mots_communs.add(mot)

                nombre_mots_commun = len(mots_communs)
                distance_tweet = (nombre_total_mots - nombre_mots_commun)/nombre_total_mots
                distances.append(distance_tweet)
            return distances
        
        if self.distance == "levenshtein":
            for autre_tweet in self.data["text"]:
                simi = levenshtein_similarity(self.tweet_a_categoriser,autre_tweet)
                distance_tweet = 1 - simi
                distances.append(distance_tweet) 
            return distances
        
        if self.distance == "lcs":
            for autre_tweet in self.data["text"]:
                simi = lcs_similarity(self.tweet_a_categoriser,autre_tweet)
                distance_tweet = 1 - simi
                distances.append(distance_tweet)
            return distances

        if self.distance == "damerau_levenshtein":
            for autre_tweet in self.data["text"]:
                simi = damerau_levenshtein_similarity(self.tweet_a_categoriser,autre_tweet)
                distance_tweet = 1 - simi
                distances.append(distance_tweet)
            return distances
        
        if self.distance == "hamming":
            for autre_tweet in self.data["text"]:
                simi = hamming_similarity(self.tweet_a_categoriser,autre_tweet)
                distance_tweet = 1 - simi
                distances.append(distance_tweet)
            return distances
        
        if self.distance == "jaro":
            for autre_tweet in self.data["text"]:
                simi = jaro_similarity(self.tweet_a_categoriser,autre_tweet)
                distance_tweet = 1 - simi
                distances.append(distance_tweet)
            return distances
            
        if self.distance == "cosine":
            for autre_tweet in self.data["text"]:
                simi = cosine_similarity(self.tweet_a_categoriser,autre_tweet)
                distance_tweet = 1 - simi
                distances.append(distance_tweet)
            return distances
        
        if self.distance == "jaccard":
            for autre_tweet in self.data["text"]:
                simi = jaccard_similarity(self.tweet_a_categoriser,autre_tweet)
                distance_tweet = 1 - simi
                distances.append(distance_tweet)
            return distances
        
        if self.distance == "sorensen_dice":
            for autre_tweet in self.data["text"]:
                simi = sorensen_dice_similarity(self.tweet_a_categoriser,autre_tweet)
                distance_tweet = 1 - simi
                distances.append(distance_tweet)
            return distances

        if self.distance == "qgram_dice":
            for autre_tweet in self.data["text"]:
                simi = qgram_similarity(self.tweet_a_categoriser,autre_tweet)
                distance_tweet = 1 - simi
                distances.append(distance_tweet)
            return distances
        
    
    def classification(self):
        label_et_distance_proches_voisins_croissant = [(x,y) for (x,y) in zip(self.data["target"],self.liste_distance())]
        label_et_distance_proches_voisins_croissant.sort(key=lambda x: x[1])
        votes = {}
        votes["4"] = 0
        votes["2"] = 0
        votes["0"] = 0
            
        if self.vote == "majoritaire":
            for i in range(0,self.nombre_voisins):
                if label_et_distance_proches_voisins_croissant[i][0] == 4:
                    votes["4"] += 1
                elif label_et_distance_proches_voisins_croissant[i][0] == 2:
                    votes["2"] += 1
                else:
                    votes["0"] += 1 
            # on sélectionne l'étiquette avec le maximum de votes
            label = ""
            vote_majoritaire = max(votes.values())
            for i in votes:
                if votes[i] == vote_majoritaire:
                    label = i #on prend en compte la dernière étiquette égale au max 
            return label
        
        if self.vote == "pondéré":
            for i in range(0,self.nombre_voisins):
                if label_et_distance_proches_voisins_croissant[i][0] == 4:
                    votes["4"] += 1/((label_et_distance_proches_voisins_croissant[i][1])**2)
                    
                elif label_et_distance_proches_voisins_croissant[i][0] == 2:
                    votes["2"] += 1/((label_et_distance_proches_voisins_croissant[i][1])**2)
                else:
                    votes["0"] += 1/((label_et_distance_proches_voisins_croissant[i][1])**2)
            # on sélectionne l'étiquette avec le maximum de votes
            label = ""
            vote_majoritaire = max(votes.values())
            for i in votes:
                if votes[i] == vote_majoritaire:
                    label = i #on prend en compte la dernière étiquette égale au max 
            return label
            

In [17]:
KNN = KNN(data_train, "how can you not love Obama? he makes jokes about himself.", 10, "levenshtein", "majoritaire")

In [12]:
KNN.classification()

'2'

In [25]:
# test avec toutes les distances, sauf hamming qui ne fonctionne que pour des phrases de mêmes longueurs
distances = ["naïve","levenshtein","lcs","damerau_levenshtein", "jaro", "cosine","jaccard","sorensen_dice","qgram_dice"]
votes = ["majoritaire", "pondéré"]

In [30]:
res = {}
for d in distances:
    res[d] = ""
    couple = []
    for v in votes:
        knn = KNN(data_train, "Fuck this economy. I hate aig and their non loan given asses.", 10, d , v)
        couple.append((v,knn.classification()))
    res[d] = couple

In [31]:
res

{'naïve': [('majoritaire', '0'), ('pondéré', '0')],
 'levenshtein': [('majoritaire', '0'), ('pondéré', '0')],
 'lcs': [('majoritaire', '0'), ('pondéré', '0')],
 'damerau_levenshtein': [('majoritaire', '0'), ('pondéré', '0')],
 'jaro': [('majoritaire', '0'), ('pondéré', '0')],
 'cosine': [('majoritaire', '0'), ('pondéré', '0')],
 'jaccard': [('majoritaire', '0'), ('pondéré', '0')],
 'sorensen_dice': [('majoritaire', '0'), ('pondéré', '0')],
 'qgram_dice': [('majoritaire', '0'), ('pondéré', '0')]}